# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem because, from the question and mission statement, we are concerned with two categorial types of students: those who need intervention, and those who do not need intervention. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [76]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [77]:
from __future__ import division
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == "yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == "no"])

# TODO: Calculate graduation rate
grad_rate = n_passed/n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [78]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

First check what data type is each column.

In [79]:
print X_all.dtypes

school        object
sex           object
age            int64
address       object
famsize       object
Pstatus       object
Medu           int64
Fedu           int64
Mjob          object
Fjob          object
reason        object
guardian      object
traveltime     int64
studytime      int64
failures       int64
schoolsup     object
famsup        object
paid          object
activities    object
nursery       object
higher        object
internet      object
romantic      object
famrel         int64
freetime       int64
goout          int64
Dalc           int64
Walc           int64
health         int64
absences       int64
dtype: object


In [80]:
print "{:20s} {:50}  {}".format("column", "unique elements", "type of element")
print "-" * 100
for col, col_d in X_all.iteritems():
    if col_d.dtype == object:
        print "{:20s}  {:50}  {}".format(col, col_d.unique(), col_d.dtype)

column               unique elements                                     type of element
----------------------------------------------------------------------------------------------------
school                ['GP' 'MS']                                         object
sex                   ['F' 'M']                                           object
address               ['U' 'R']                                           object
famsize               ['GT3' 'LE3']                                       object
Pstatus               ['A' 'T']                                           object
Mjob                  ['at_home' 'health' 'other' 'services' 'teacher']   object
Fjob                  ['teacher' 'other' 'services' 'health' 'at_home']   object
reason                ['course' 'other' 'home' 'reputation']              object
guardian              ['mother' 'father' 'other']                         object
schoolsup             ['yes' 'no']                                        object


### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [81]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


Inspect the data after the transformation

In [82]:
print "{:20s} {:50}  {}".format("column", "unique elements", "type of element")
print "-" * 100
for col, col_d in X_all.iteritems():
    print "{:20s} : {:50}  {}".format(col, col_d.unique(), col_d.dtype)

column               unique elements                                     type of element
----------------------------------------------------------------------------------------------------
school_GP            : [ 1.  0.]                                           float64
school_MS            : [ 0.  1.]                                           float64
sex_F                : [ 1.  0.]                                           float64
sex_M                : [ 0.  1.]                                           float64
age                  : [18 17 15 16 19 22 20 21]                           int64
address_R            : [ 0.  1.]                                           float64
address_U            : [ 1.  0.]                                           float64
famsize_GT3          : [ 1.  0.]                                           float64
famsize_LE3          : [ 0.  1.]                                           float64
Pstatus_A            : [ 1.  0.]                                 

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [83]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size = num_train)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

1. Ensemble Method, Random Forest
 * *Usage*: One usage I could find for decision trees is in the [video game industry](https://www.quora.com/What-companies-or-industries-use-random-forests-for-data-analysis).
 * *Strengths*: Like most ensemble methods, the random forest approach enhances our chances in avoiding overfitting. A random forest avoids overfitting by having many 'weak learner' decision trees vote on a particular classification for a data point.
 * *Weaknesses*: Unlike a single decision tree where the results are interpretable from looking at the decision path that the tree took, a random forest is less interpretable because its result is a mixture of multiple decisoin trees. Fruthermore, it can take a long time to test/predict a single data point instance, since many decision trees need to be run and their results voted on.
 * I believe this is a good model to use because most of our features are categorical and can therefore have a simple decision line separation between them. Since decision trees are good at making decision boundaries between categorial data, we can use random forests to have many decision trees and reduce their chances of overfitting to the training data by relying on the enemble method. This algorithm would be a poor choice if the features were mostly continously numerical in nature.
 
2. Gaussian Naive Bayes
 * *Usage*: [Naive Bayes could used in spam detection and classifying a written piece/webpage according to topics.](https://www.quora.com/In-what-real-world-applications-is-Naive-Bayes-classifier-used) Their usage in Natural Language Processing is driven by the probabilistic nature of language assumptions (the probability of 
 * *Strengths*: This is a simple machine learning model, which means that the assumptions used are not beyond reason, and that the results and predictions of the model are easily interpretable.
 * *Weaknesses*: This model requires a large, representative sample of the population to be able to provide accurate predictions to unseen data. This is due to the statistical nature of this model. A larger sample of data points gives us a more confident representation of the original population.
 * I believe this is a good model to use because the absense or presence of a (categorial) feature should add to a statistical measure of how much a student needs intervention. If enough of these features add up to above a threshold, then the model should be predict that a student needs intervention.
 
3. Decision Tree
 * *Usage*: Decision trees are used in Natural Language Processing and loan decision in banks, among other uses.
 * *Strengths*: Decision trees are a simple and intuitive algorithm whose answers can be interpreted rather easily by following the decision line that the tree took to reach its conclusion.
 * *Weaknesses*: Decisions trees are prone to overfitting. Pruning is one procedure to reduce the chances of overfitting. It works by removing leaf decisions that fail to reach a certain information gain threshold. However, pruning leads to a worsening of training error, and can not guarantee an increase in performance in test accuracy.
 * I belive this is a good model to ues because most of the data that we have is categorial, which lends itself well to this algorithm. Furthermore, the decision to have an intervention can be done by deciding on each category and then seeing what options we are left with, which is exactly what a decision tree does.
    

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [84]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [85]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.ensemble import RandomForestClassifier
# from sklearn import model_B
from sklearn.naive_bayes import GaussianNB
# from skearln import model_C
from sklearn.tree import DecisionTreeClassifier
# TODO: Initialize the three models
clf_A = RandomForestClassifier()
clf_B = GaussianNB()
clf_C = DecisionTreeClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    for X_train, y_train in [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]:
        train_predict(clf, X_train, y_train, X_test, y_test)
        print "-"*100
    print "#" *100

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0364 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.9857.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.7939.
----------------------------------------------------------------------------------------------------
Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0252 seconds
Made predictions in 0.0019 seconds.
F1 score for training set: 0.9927.
Made predictions in 0.0014 seconds.
F1 score for test set: 0.7385.
----------------------------------------------------------------------------------------------------
Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0353 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.9926.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.6992.
------------------------------------------------------------------------

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

### Note: The table reflects ou

** Classifer 1 - Random Forests**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0323        |      0.0018           |     0.9833        |       0.6496        |
| 200               | 0.0271          |      0.0017           |     0.9923        |       0.6829        |
| 300               | 0.0259          |      0.0025           |     0.9900        |       0.7576        |

** Classifer 2 - Guassian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0012          |      0.0010           |     0.7903        |       0.7018        |
| 200               | 0.0013          |      0.0005           |     0.7912        |       0.7680        |
| 300               | 0.0021          |      0.0006           |     0.8083        |       0.7656        |

** Classifer 3 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0017          |      0.0003           |     1.0000        |       0.5664        |
| 200               | 0.0021          |      0.0002           |     1.0000        |       0.7317        |
| 300               | 0.0024          |      0.0002           |     1.0000        |       0.6667        |


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** I would highly recommend the Gaussian Naive Bayes Classifier (GNB) in this case. The GNB performs very favorably amongst other classifiers, providing both a higher testing F1 score, and a faster testing time than most other classifiers. The model is also fairly fast to train, faster than both decision tree and random forest classifiers. It also appears to not overfit the data, as can be seen from the training set F1 score(0.79-0.81), while the Decision Tree and Random forest implementations tended to overfit the training data as their F1 score on the training set was almost perfect (0.9833-1.000).  This results in a higher F1 score performance for the GNB (0.7-0.76) than both the Decisoin Tree (0.57-0.67) and the Random Forest (0.65-0.76) classifiers.

Although the Random Forest classifier comes close to the score of the GNB, I would not recommend attempting to improve the Random Forest classifier because it is easy to see how it is greatly overfitting the data by notivcing the F1 score during trainning (0.98-0.99). 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

### Parameters to Tune
    
  * ~~The only parameter that is available at construction time for the GNB classifier is the prior parameter, which specifies the prior probabilities of each class in the dataset. In our case, the priors are the probabiliteis of each class. These are calculated below.~~
  * ~~These are the probabilities for the training+testing set, so our algorithm actually doesn't know all this when it is training.~~
  * ~~It would be interesting to see what happens when we give the algorithm a list of priors that deviate from this distribution.~~


In [86]:
print """"Prior probability of class "yes" for intervention: {}""".format((y_all[y_all=="no"]).count()/y_all.count())
print """"Prior probability of class "no" for intervention: {}""".format((y_all[y_all=="yes"]).count()/y_all.count())

"Prior probability of class "yes" for intervention: 0.329113924051
"Prior probability of class "no" for intervention: 0.670886075949


In [90]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from  sklearn import grid_search
from sklearn.metrics import make_scorer, f1_score
# TODO: Create the parameters list you wish to tune
parameters = {'priors':[[.1,.9], [.2,.8], [.3,.9], [.4,.6], [.5,.5], [.6,.4], [.7,.3], [.8,.2], [.9,.1]] }

#ERROR: this is where the error happens, indicating that the gaussian Naive Bayes implementation here does not
#take any parameter. This code is meant for demonstration and testing purpose.
# TODO: Initialize the classifier
clf = GaussianNB([0.1,.9])

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = f1_score

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

ValueError: Invalid parameter priors for estimator GaussianNB. Check the list of available parameters with `estimator.get_params().keys()`.

### Parameters to Tune (continued)
  * Unfortunately, although [documentation says otherwise](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html), the Gaussian Naive Bayes classifier implementation in sklearn does not accept any parameters at construction time, making it inaccessible to fine-tunning. Therefore, we will use our second best optimizer, the random forest classifier to fine tune. If this fine-tunned classifier performs better than GNB, we will report it as the best classifier.
  * Fine tunning the random forest: There are many parameters that can be fine tunned with the random forst approach. The following is a list of the ones that are tuned blow:
    * criterion: this decides which split criterion to follow. As we learned from Michael and Chris, entropy is about information gain, while gini works in another, but efficient, way.
    * max_features: this controls the maximum number of features that each decision tree in the forst can leverage at any given time when splitting the data. The functions passes calculate the number of maximum features from the number of total features given. This helps increase the speed performance of the trees by limiting how many features to calculate split gain for.
    * max_depth: this limits the number of splits allowed in each decision tree. This is used to control overfitting!
    * min_samples_split: in this context, passing a floating point indicates passing a probabilty that must be satisfied (equal or higher) when splitting the input at any given node.
  * We run the grid search 5 times to observe the variation in the result

In [127]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer, f1_score

#run 5 times to see variation
for trial in range(5):
    # TODO: Create the parameters list you wish to tune
    parameters = {'criterion': ['gini', 'entropy'], \
                  'max_features': ['sqrt', 'log2', 'auto'], \
                  'max_depth': [1,2,3,4,5,6,7,8, None], \
                 'min_samples_split': [0.2, 0.3, 0.4, 0.7]}

    #ERROR: this is where the error happens, indicating that the gaussian Naive Bayes implementation here does not
    #take any parameter. This code is meant for demonstration and testing purpose.
    # TODO: Initialize the classifier
    clf = RandomForestClassifier()

    # TODO: Make an f1 scoring function using 'make_scorer' 
    f1_scorer = make_scorer(f1_score, pos_label="yes")

    # TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
    grid_obj = grid_search.GridSearchCV(clf, parameters, scoring = f1_scorer)

    # TODO: Fit the grid search object to the training data and find the optimal parameters
    grid_obj.fit(X_train, y_train)

    # Get the estimator
    clf = grid_obj.best_estimator_

    print "-"*100
    print "Trial #",trial+1
    # Report the final F1 score for training and testing after parameter tuning
    print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
    print "The best estimator has the following parameters:"
    for k, v in clf.get_params().iteritems():
        if k in parameters:
            print "{:30}  {:>10}".format(k, v)

----------------------------------------------------------------------------------------------------
Trial # 1
Made predictions in 0.0013 seconds.
Tuned model has a training F1 score of 0.8048.
Made predictions in 0.0008 seconds.
Tuned model has a testing F1 score of 0.8000.
The best estimator has the following parameters:
min_samples_split                      0.7
criterion                             gini
max_features                          sqrt
max_depth                                2
----------------------------------------------------------------------------------------------------
Trial # 2
Made predictions in 0.0012 seconds.
Tuned model has a training F1 score of 0.8073.
Made predictions in 0.0008 seconds.
Tuned model has a testing F1 score of 0.8052.
The best estimator has the following parameters:
min_samples_split                      0.2
criterion                          entropy
max_features                          auto
max_depth                                2
------

### Question 5 - Final F<sub>1</sub> Score
The fine tunned model performs much better than the untuned model. It tends to perform about 10% better on F1 score than the untuned model. However, it seems that there is variation in the results of what is the best parameter for min_samples_split and max_features numberes. Since max_features is more geared towards speed performance, we disregard this, since the optimization is constrained by the f1 score, not the speed. However, the min_samples_split parameter controls the threshold for which to make more splits in the data. It shows a range of variation from 0.3 to 0.7. This is an interesting side effect of the optimization.

It is worthy to note that the fine tuned random forst algorithm outpreformed the GNB implementation.

**Answer: **

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.